In [ ]:
!pip install --upgrade datasets
!pip install py7zr

!pip install regex
import regex as re

!pip install rouge
from rouge import Rouge

import pandas as pd

from datasets import load_dataset
dataset = load_dataset('samsum')

     |████████████████████████████████| 225kB 4.0MB/s 
     |████████████████████████████████| 245kB 38.9MB/s 
     |████████████████████████████████| 112kB 45.1MB/s 
     |████████████████████████████████| 71kB 2.5MB/s 
     |████████████████████████████████| 1.9MB 5.6MB/s 
     |████████████████████████████████| 122kB 37.8MB/s 
     |████████████████████████████████| 2.2MB 37.5MB/s 
     |████████████████████████████████| 358kB 38.8MB/s 


Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


In [ ]:
dataset["train"][0]['dialogue'].replace('\r', ' ').replace('\n', '')

"Amanda: I baked  cookies. Do you want some? Jerry: Sure! Amanda: I'll bring you tomorrow :-)"

### Creating test_df dataframe

In [ ]:
test_df = pd.DataFrame(data={"dialogue": dataset["test"][:]['dialogue'], "summary": dataset["test"][:]['summary']})

def removeNewline(text):
    return text.replace('\r', ' ').replace('\n', '')
    
test_df.dialogue = test_df.dialogue.apply(lambda x: removeNewline(x))
test_df.summary = test_df.summary.apply(lambda x: removeNewline(x))

In [ ]:
test_df['dialogue'][0]
test_df['summary'][0]

"Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry."

In [ ]:
# printing an example from test_df

print(test_df['dialogue'][100])
print(test_df['summary'][100])

Chris: Hi there! Where are you? Any chance of skyping? Rick: Hi! Our last two days in Cancun before flying to Havana. Yeah, skyping is an idea. When would it suit you? Rick: We don't have the best of connections in the room but I can get you pretty well in the lobby. Chris: What's the time in your place now? Rick: 6:45 pm Chris: It's a quarter to one in the morning here. Am still in front of the box. Rick: Gracious me! Sorry mate. You needn't have answered. Chris: 8-D Rick: Just tell me when we could skype. Chris: Preferably in the evening. Just a few hours earlier than now. And not tomorrow. Rick: Shute! Only tomorrow makes sense as there's no workable internet in Cuba. Chris: Could you make it like 3 pm your time? Rick: Sure. Chris: Perfect. So talk to you tomorrow. Chris: Give my love to Helen please. Rick: I will. Thx.
Rick and Helen are in Cancun. They're flying to Havana in two days. Chris and Rick will talk on Skype at 3 PM in Mexico.


### Splitting the df_test by .?! signs

In [ ]:
# trying out the splitting by .!? signs, on one example

print(test_df['dialogue'][40])
a = re.split('(?<=[.!?]) +',test_df['dialogue'][40])
a

Sebastian: It's been already a year since we moved here. Sebastian: This is totally the best time of my life. Kevin: Really?  Sebastian: Yeah! Totally maaan. Sebastian: During this 1 year I learned more than ever.  Sebastian: I learned how to be resourceful, I'm learning responsibility, and I literally have the power to make my dreams come true. Kevin: It's great to hear that. Kevin: It's great that you are satisfied with your decisions. Kevin: And above all it's great to see that you have someone you love by your side :) Sebastian: Exactly! Sebastian: That's another part of my life that is going great. Kevin: I wish I had such a person by my side. Sebastian: Don't worry about it. Sebastian: I have a feeling this day will come shortly. Kevin: Haha. I don' think so, but thanks. Sebastian: This one year proved to me that when you want something really badly, you can achieve it. Kevin: I want to win lottery and I never did :D Sebastian: If you devoted your life to analyze all of the winni

["Sebastian: It's been already a year since we moved here.",
 'Sebastian: This is totally the best time of my life.',
 'Kevin: Really?',
 'Sebastian: Yeah!',
 'Totally maaan.',
 'Sebastian: During this 1 year I learned more than ever.',
 "Sebastian: I learned how to be resourceful, I'm learning responsibility, and I literally have the power to make my dreams come true.",
 "Kevin: It's great to hear that.",
 "Kevin: It's great that you are satisfied with your decisions.",
 "Kevin: And above all it's great to see that you have someone you love by your side :) Sebastian: Exactly!",
 "Sebastian: That's another part of my life that is going great.",
 'Kevin: I wish I had such a person by my side.',
 "Sebastian: Don't worry about it.",
 'Sebastian: I have a feeling this day will come shortly.',
 'Kevin: Haha.',
 "I don' think so, but thanks.",
 'Sebastian: This one year proved to me that when you want something really badly, you can achieve it.',
 'Kevin: I want to win lottery and I never di

In [ ]:
a = re.split('(?<=[.!?]) +',test_df['dialogue'][40])
print(len(a))
summary_ex=[]
for i in range(0,3):
  summary_ex.append(a[i])

print(summary_ex)

# joining the splitted parts of the dialogue into one "list", on one example

summary_ex = ''.join(summary_ex)
summary_ex

# summary provided by the dataset
summary_gen = test_df['summary'][40]

22
["Sebastian: It's been already a year since we moved here.", 'Sebastian: This is totally the best time of my life.', 'Kevin: Really?']


In [ ]:
# getting the Rouge score for the example

rouge = Rouge()
rouge.get_scores(summary_ex, summary_gen)

[{'rouge-1': {'f': 0.05714285247346978,
   'p': 0.045454545454545456,
   'r': 0.07692307692307693},
  'rouge-2': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'rouge-l': {'f': 0.060606055977961794,
   'p': 0.047619047619047616,
   'r': 0.08333333333333333}}]

### Extractive model for 3 first sentences

In [ ]:
summary_ex=[]
for i in range(0,len(test_df)):
    #print(i)
    a = re.split('(?<=[.!?]) +',test_df['dialogue'][i])
    summary_ex_inter=[]
    if len(a)>=3:
      for i in range(0,3):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex.append(summary_ex_inter)
    else:
      for i in range(0,len(a)):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex.append(summary_ex_inter)
print(summary_ex[40])
len(summary_ex)

Sebastian: It's been already a year since we moved here.Sebastian: This is totally the best time of my life.Kevin: Really?


819

In [ ]:
print(summary_ex[20])

Deirdre: Hi Beth, how are you love?Beth: Hi Auntie Deirdre, I'm been meaning to message you, had a favour to ask.Deirdre: Wondered if you had any thought about your Mum's 40th, we've got to do something special!


In [ ]:
# summary provided by the dataset

summary_gen = []
for i in range(0,len(test_df)):
  summary_gen.append(test_df['summary'][i])
len(summary_gen)

819

In [ ]:
# ROUGE score for the first 3 sentences model

rouge = Rouge()
rouge.get_scores(summary_ex, summary_gen, avg=True)

{'rouge-1': {'f': 0.17098705460213398,
  'p': 0.1530879089239828,
  'r': 0.258195076655181},
 'rouge-2': {'f': 0.060451162392397714,
  'p': 0.05390675191117815,
  'r': 0.09235748207626746},
 'rouge-l': {'f': 0.1665168969037099,
  'p': 0.14917296489214862,
  'r': 0.23530866438577852}}

### Extractive model for x first sentences. x chosen as 7

In [ ]:
x = 7 # random number !
summary_ex_chosennbr=[]
for i in range(0,len(test_df)):
    #print(i)
    a = re.split('(?<=[.!?]) +',test_df['dialogue'][i])
    summary_ex_inter=[]
    if len(a)>=x:
      for i in range(0,x):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex_chosennbr.append(summary_ex_inter)
    else:
      for i in range(0,len(a)):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex_chosennbr.append(summary_ex_inter)
print(summary_ex_chosennbr[40])
len(summary_ex_chosennbr)

Sebastian: It's been already a year since we moved here.Sebastian: This is totally the best time of my life.Kevin: Really?Sebastian: Yeah!Totally maaan.Sebastian: During this 1 year I learned more than ever.Sebastian: I learned how to be resourceful, I'm learning responsibility, and I literally have the power to make my dreams come true.


819

In [ ]:
# ROUGE score for Extractive model for x first sentences

rouge = Rouge()
rouge.get_scores(summary_ex_chosennbr, summary_gen, avg=True)

{'rouge-1': {'f': 0.1897774707001165,
  'p': 0.13997869945476335,
  'r': 0.3596000840038394},
 'rouge-2': {'f': 0.06921871574663098,
  'p': 0.0511823644578453,
  'r': 0.12988007954210956},
 'rouge-l': {'f': 0.1881871465175068,
  'p': 0.14122095850596653,
  'r': 0.32681766569419624}}

In [ ]:
import random

### Extractive model for x random sentences

In [ ]:
x = 3 # random number !
summary_ex_chosennbr_randomsent=[]
for i in range(0,len(test_df)):
    #print(i)
    a = re.split('(?<=[.!?]) +',test_df['dialogue'][i])
    #print(a)
    summary_ex_inter=[]
    if len(a)>=x:
      random.seed(9)
      a = random.sample(a, x)
      #print(a)
      for i in range(0,x):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex_chosennbr_randomsent.append(summary_ex_inter)
    else:
      for i in range(0,len(a)):
        summary_ex_inter.append(a[i])
      summary_ex_inter = ''.join(summary_ex_inter)
      summary_ex_chosennbr_randomsent.append(summary_ex_inter)
print(summary_ex_chosennbr_randomsent[40])
len(summary_ex_chosennbr_randomsent)

Kevin: Haha.Sebastian: Something like that xD Kevin: I'm happy for you man.Kevin: I wish I had such a person by my side.


819

In [ ]:
# ROUGE score for Extractive model for x random sentences

rouge = Rouge()
rouge.get_scores(summary_ex_chosennbr_randomsent, summary_gen, avg=True)

{'rouge-1': {'f': 0.18736646441733382,
  'p': 0.13710884091371625,
  'r': 0.3576382796708569},
 'rouge-2': {'f': 0.06719862837151498,
  'p': 0.04922512972157836,
  'r': 0.12767151510226057},
 'rouge-l': {'f': 0.18520875867671407,
  'p': 0.13801851907804008,
  'r': 0.3242882354604086}}